In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set()

In [2]:
df = pd.read_csv('Churn_Modelling.csv', index_col = 'RowNumber')
df.head()


,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
RowNumber,,,,,,,,,,,,,
1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
X_columns = df.columns.tolist()[2:12]
Y_columns = df.columns.tolist()[-1:]
print(X_columns)
print(Y_columns)

['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']
['Exited']


In [4]:
X = df[X_columns].values 
Y = df[Y_columns].values

In [5]:
from sklearn.preprocessing import LabelEncoder
X_column_transformer = LabelEncoder()
X[:, 1] = X_column_transformer.fit_transform(X[:, 1])

In [6]:
X[:, 2] = X_column_transformer.fit_transform(X[:, 2])

We are treating countries with ordinal values(0 < 1 < 2) but they are incomparable.
To solve this we can use one hot encoding.
We will perform some standardization 

In [7]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

pipeline = Pipeline(
    [
        ('Categorizer', ColumnTransformer(
            [
                ("Gender Label Encoder", OneHotEncoder(categories = 'auto', drop = 'first'), [2]),
                ("Geography Label Encoder", OneHotEncoder(categories = 'auto', drop = 'first'), [1])
            ], 
            remainder = 'passthrough', n_jobs = 1)),
        ('Normalizer', StandardScaler())
    ]
)

In [8]:
X = pipeline.fit_transform(X)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [10]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [11]:
classifier = Sequential()

In [12]:
classifier.add(Dense(6, activation = 'relu', input_shape = (X_train.shape[1], )))
classifier.add(Dropout(rate = 0.1))

In [13]:
classifier.add(Dense(6, activation = 'relu'))
classifier.add(Dropout(rate = 0.1))

In [14]:
classifier.add(Dense(1, activation = 'sigmoid'))

In [15]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 72        
                                                                 
 dropout (Dropout)           (None, 6)                 0         
                                                                 
 dense_1 (Dense)             (None, 6)                 42        
                                                                 
 dropout_1 (Dropout)         (None, 6)                 0         
                                                                 
 dense_2 (Dense)             (None, 1)                 7         
                                                                 
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [16]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
history = classifier.fit(X_train, y_train, batch_size = 32, epochs = 200, validation_split = 0.1, verbose = 2)

Epoch 1/200
225/225 - 5s - loss: 0.6305 - accuracy: 0.7140 - val_loss: 0.5356 - val_accuracy: 0.7962 - 5s/epoch - 20ms/step
Epoch 2/200
225/225 - 1s - loss: 0.5071 - accuracy: 0.7990 - val_loss: 0.4537 - val_accuracy: 0.8037 - 892ms/epoch - 4ms/step
Epoch 3/200
225/225 - 1s - loss: 0.4689 - accuracy: 0.7997 - val_loss: 0.4317 - val_accuracy: 0.8075 - 866ms/epoch - 4ms/step
Epoch 4/200
225/225 - 1s - loss: 0.4500 - accuracy: 0.8064 - val_loss: 0.4233 - val_accuracy: 0.8100 - 857ms/epoch - 4ms/step
Epoch 5/200
225/225 - 1s - loss: 0.4484 - accuracy: 0.8064 - val_loss: 0.4202 - val_accuracy: 0.8075 - 866ms/epoch - 4ms/step
Epoch 6/200
225/225 - 1s - loss: 0.4426 - accuracy: 0.8076 - val_loss: 0.4167 - val_accuracy: 0.8138 - 874ms/epoch - 4ms/step
Epoch 7/200
225/225 - 1s - loss: 0.4378 - accuracy: 0.8108 - val_loss: 0.4113 - val_accuracy: 0.8200 - 871ms/epoch - 4ms/step
Epoch 8/200
225/225 - 1s - loss: 0.4355 - accuracy: 0.8103 - val_loss: 0.4072 - val_accuracy: 0.8213 - 887ms/epoch - 4ms

In [18]:
y_pred = classifier.predict(X_test)
print(y_pred[:5])

63/63 [==============================] - 0s 2ms/step
[[0.25835028]
 [0.28194377]
 [0.18433088]
 [0.07922497]
 [0.08735227]]


In [19]:
y_pred = (y_pred > 0.5).astype(int)
print(y_pred[:5])

[[0]
 [0]
 [0]
 [0]
 [0]]


In [20]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1551   44]
 [ 227  178]]


In [21]:
print(((cm[0][0] + cm[1][1])* 100) / len(y_test), '% of data was classified correctly')

86.45 % of data was classified correctly
